In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from visual_behavior import database as db
from allensdk.brain_observatory.behavior.behavior_ophys_session import BehaviorOphysSession
import seaborn as sns
from visual_behavior.ophys.response_analysis.response_analysis import ResponseAnalysis
import visual_behavior.ophys.response_analysis.response_processing as rp
import visual_behavior.data_access.loading as loading

from multiprocessing import Pool
%widescreen

/home/dougo/Code/AllenSDK/allensdk/brain_observatory/behavior/behavior_ophys_api/behavior_ophys_nwb_api.py:9: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [2]:
sns.set_context('notebook',font_scale =1.5)

In [3]:
from dro.modules import plotting as plotting
import visual_behavior.visualization.ophys.summary_figures as sf

In [4]:
# convert 'flash_metrics_label' to 'engagement_state'
def assign_engagement_label(input_label):
    if 'high-lick' in input_label:
        return 'engaged'
    elif 'low-lick' in input_label:
        return 'disengaged'

In [5]:
def load_df(dataset, df_name):
    query = '''
        select bs.id
        from behavior_sessions bs
        join ophys_experiments oe on oe.ophys_session_id = bs.ophys_session_id
        where oe.id = {}
    '''
    behavior_session_id = db.lims_query(query.format(dataset.experiment_id))
    
    cache = loading.get_visual_behavior_cache()
    experiments_table = loading.get_filtered_ophys_experiment_table()

    analysis = ResponseAnalysis(dataset, overwrite_analysis_files=True) 
    
    sdf = analysis.get_response_df(df_name='stimulus_response_df')
    
    flash_metrics = loading.get_stim_metrics_summary(behavior_session_id,load_location='from_database')
    flash_metrics = flash_metrics.drop(columns=['start_time']).merge(
        sdf[['stimulus_presentations_id','start_time']].drop_duplicates(),
        left_on = 'flash_index',
        right_on = 'stimulus_presentations_id',
        how='left'
    )
    
    merge_columns = {
        'stimulus_response_df':{
            'left':'stimulus_presentations_id',
            'right':'flash_index',
        }, 
        'omission_response_df':{
            'left':'stimulus_presentations_id',
            'right':'flash_index',
        }, 
        'trials_response_df':{
            'left':'change_time',
            'right':'start_time',
        }, 
    }
    
    
    df = analysis.get_response_df(df_name=df_name, format='long')
    df = df.merge(
        flash_metrics,
        left_on = merge_columns[df_name]['left'],
        right_on = merge_columns[df_name]['right'],
        how = 'left'
    )
    df['engagement_state'] = df['flash_metrics_labels'].map(lambda x: assign_engagement_label(x))
    return df

In [13]:
def seaborn_plot(df,ax,cell_id):
    sns.lineplot(
        x = 'eventlocked_timestamps',
        y = 'eventlocked_traces',
        data = df.query('cell_specimen_id == @cell_id'),
        hue='engagement_state',
        ax=ax,
    )
    
def make_plots(dataset):
    figure_savedir = '/allen/programs/braintv/workgroups/nc-ophys/visual_behavior/summary_plots/single_cell_plots/response_plots'
    titles = [
        'stimulus response',
        'omission response',
        'change response'
    ]
    dfs = {}
    for ii,df_name in enumerate(['stimulus_response_df', 'omission_response_df', 'trials_response_df']):
        dfs[df_name] = load_df(dataset,df_name)
        
    oeid = dataset.ophys_experiment_id
    for cell_id in dfs['omission_response_df']['cell_specimen_id'].unique():
        fig,ax = plt.subplots(1,3,figsize=(20,7),sharey=True)
        for ii,df_name in enumerate(['stimulus_response_df', 'omission_response_df', 'trials_response_df']):
            ax[ii].cla()
            df = dfs[df_name]
            seaborn_plot(df,ax[ii],cell_id)

            if df_name == 'stimulus_response_df':
                plotting.designate_flashes(ax[ii])
                ax[ii].set_xlim(-0.5,0.75)
            elif df_name == 'omission_response_df':
                plotting.designate_flashes(ax[ii], omit=0)
                ax[ii].set_xlim(-3,3)
            elif df_name == 'trials_response_df':
                plotting.designate_flashes(ax[ii], pre_color='black', post_color='green')
                ax[ii].set_xlim(-3,3)
            ax[ii].set_xlabel('time (s)')
            if ii == 0:
                ax[ii].set_ylabel('$\Delta$F/F')

        fig.tight_layout()
        plt.subplots_adjust(top=0.9)
        title = get_title(oeid,cell_id,experiments_table)
        fig.suptitle(title)
        fig.savefig(os.path.join(figure_savedir,title+'.png'),dpi=200)

In [14]:
def get_title(oeid,cell_specimen_id,experiments_table):
    row = experiments_table.query('ophys_experiment_id == @oeid').iloc[0].to_dict()
    title = '{}__specimen_id={}__exp_id={}__{}__{}__depth={}__cell_id={}'.format(
        row['cre_line'],
        row['specimen_id'],
        row['ophys_experiment_id'],
        row['session_type'],
        row['targeted_structure'],
        row['imaging_depth'],
        cell_specimen_id,
    )
    return title

In [15]:
def generate_save_plots(experiment_id):
    cache_dir = r'\\allen\programs\braintv\workgroups\nc-ophys\visual_behavior\visual_behavior_production_analysis'.replace('\\','/')
    dataset = loading.get_sdk_dataset(experiment_id, cache_dir)
    make_plots(dataset)

In [16]:
import visual_behavior.data_access.loading as loading
cache = loading.get_visual_behavior_cache()
experiments_table = loading.get_filtered_ophys_experiment_table()

In [17]:
def load_flashwise_summary(behavior_session_id=None):
    conn = db.Database('visual_behavior_data')
    collection = conn['behavior_analysis']['annotated_stimulus_presentations']

    if behavior_session_id is None:
        # load all
        df = pd.DataFrame(list(collection.find({})))
    else:
        # load data from one behavior session
        df = pd.DataFrame(list(collection.find({'behavior_session_id':int(behavior_session_id)})))

    conn.close()
    
    return df.sort_values(by=['behavior_session_id','flash_index'])

In [18]:
%%time
flash_summary = load_flashwise_summary()

CPU times: user 16.5 s, sys: 1.01 s, total: 17.5 s
Wall time: 22 s


In [19]:
experiments_table['in_flash_summary'] = experiments_table['behavior_session_id'].map(lambda bsid: bsid in list(flash_summary['behavior_session_id'].unique()))

experiments_table['in_flash_summary'].value_counts()

False    1531
True      110
Name: in_flash_summary, dtype: int64

In [ ]:
for experiment_id in experiments_table.query('in_flash_summary == True')['ophys_experiment_id']:
    generate_save_plots(experiment_id)

Sync photodiode error needs to be fixed. Using assumed monitor delay: 0.0351
/home/dougo/Code/AllenSDK/allensdk/brain_observatory/sync_dataset.py:109: UserWarning:

The loaded sync file contains the following deprecated line label keys: {'cam1_exposure', 'cam2_exposure'}. Consider updating the sync file line labels.

/home/dougo/Code/AllenSDK/allensdk/brain_observatory/sync_dataset.py:109: UserWarning:

The loaded sync file contains the following deprecated line label keys: {'cam1_exposure', 'cam2_exposure'}. Consider updating the sync file line labels.

/home/dougo/.conda/envs/visual_behavior/lib/python3.7/site-packages/xarray/core/nanops.py:142: RuntimeWarning:

Mean of empty slice

/home/dougo/Code/visual_behavior_analysis/visual_behavior/ophys/response_analysis/response_processing.py:236: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://

saving stimulus_response_df


/home/dougo/.conda/envs/visual_behavior/lib/python3.7/site-packages/pandas/core/generic.py:2505: PerformanceWarning:


your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->Index(['trace', 'trace_timestamps'], dtype='object')]




saving stimulus_response_df
saving stimulus_response_df
saving omission_response_df
saving stimulus_response_df


/home/dougo/Code/visual_behavior_analysis/visual_behavior/ophys/response_analysis/response_processing.py:236: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/home/dougo/Code/visual_behavior_analysis/visual_behavior/ophys/response_analysis/response_processing.py:290: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



...

saving stimulus_response_df
saving stimulus_response_df
saving omission_response_df
saving stimulus_response_df


/home/dougo/Code/visual_behavior_analysis/visual_behavior/ophys/response_analysis/response_processing.py:236: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/home/dougo/Code/visual_behavior_analysis/visual_behavior/ophys/response_analysis/response_processing.py:290: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



saving trials_response_df


/home/dougo/.conda/envs/visual_behavior/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning:

More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).



MemoryError: 